In [1]:
install.packages("forestploter")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(ggplot2)
library(pheatmap)
library(patchwork)
library(edgeR)
library(pROC)
library(survminer)
library(survival)
library(forestmodel)
library(forestploter)
library(grid)

Loading required package: limma

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var


Loading required package: ggpubr


Attaching package: ‘survival’


The following object is masked from ‘package:survminer’:

    myeloma




In [3]:
icb_gsdb <- lapply(read.csv("Data/gene_set.csv"), na.omit)

In [4]:
study <- list(
    Melanoma_Advanced_GSE91061_Riaz2017 = list(exp = read.csv("Data/Melanoma_Advanced_GSE91061_Riaz2017_Log2FPKM.csv", row.names = 1), pheno = read.csv("Data/Melanoma_Advanced_GSE91061_Riaz2017_pheno.csv")),
    Melanoma_Metastatic_GSE78220_Hugo2016 = list(exp = read.csv("Data/Melanoma_Metastatic_GSE78220_Hugo2016_Log2FPKM.csv", row.names = 1), pheno = read.csv("Data/Melanoma_Metastatic_GSE78220_Hugo2016_pheno.csv")),
    Melanoma_Metastatic_PRJEB23709_Gide2019 = list(exp = read.csv("Data/Melanoma_Metastatic_PRJEB23709_Gide2019_Log2FPKM.csv", row.names = 1), pheno = read.csv("Data/Melanoma_Metastatic_PRJEB23709_Gide2019_pheno.csv")),
    STAD_Metastatic_PRJEB25780_Kim2018 = list(exp = read.csv("Data/STAD_Metastatic_PRJEB25780_Kim2018_Log2FPKM.csv", row.names = 1), pheno = read.csv("Data/STAD_Metastatic_PRJEB25780_Kim2018_pheno.csv")),
    NSCLC_Advanced_GSE135222_Kim2020 = list(exp = read.csv("Data/NSCLC_Advanced_GSE135222_Kim2020_Log2TPM.csv", row.names = 1), pheno = read.csv("Data/NSCLC_Advanced_GSE135222_Kim2020_pheno.csv")),
    NSCLC_Metastatic_GSE136961_Hwang2020 = list(exp = read.csv("Data/NSCLC_Metastatic_GSE136961_Hwang2020_Log2TPM.csv", row.names = 1), pheno = read.csv("Data/NSCLC_Metastatic_GSE136961_Hwang2020_pheno.csv")),
    SU2C = list(exp = read.csv("Data/SU2C_Log2TPM.csv", row.names = 1), pheno = read.csv("Data/SU2C_pheno.csv"))
)

for(pname in names(study)){
    rownames(study[[pname]]$pheno) <- study[[pname]]$pheno$sid
}

In [7]:
# compute z-scores for gene sets

for(pname in names(study)){
    cat(pname,"\n")
    
    ngene_ovlp <- sapply(icb_gsdb, function(x){length(intersect(rownames(study[[pname]]$exp), x))})
    valid_gs <- names(ngene_ovlp)[ngene_ovlp > 0]
    
    temp <- data.frame(matrix(NA, ncol=ncol(study[[pname]]$exp), nrow=length(valid_gs)))
    rownames(temp) <- valid_gs
    colnames(temp) <- colnames(study[[pname]]$exp)

    for(the_gs in rownames(temp)){
        genes <- intersect(icb_gsdb[[the_gs]], rownames(study[[pname]]$exp))
        if(length(genes) == 1){
            temp[the_gs, ] <- as.numeric(scale(as.numeric(study[[pname]]$exp[genes,])))
        } else {
            temp[the_gs, ] <- rowMeans(scale(t(study[[pname]]$exp[genes,])))
        }
    }
    
    if(any(is.na(temp))){
        temp[is.na(temp)] <- 0
    }
    
    if(any(apply(temp == 0, 1, all))){
        temp <- temp[-which(apply(temp == 0, 1, all)),]
    }
    
    study[[pname]]$gs_zscore <- temp
    
    study[[pname]]$pheno <- cbind(study[[pname]]$pheno, t(temp))
}

Melanoma_Advanced_GSE91061_Riaz2017 
Melanoma_Metastatic_GSE78220_Hugo2016 
Melanoma_Metastatic_PRJEB23709_Gide2019 
STAD_Metastatic_PRJEB25780_Kim2018 
NSCLC_Advanced_GSE135222_Kim2020 
NSCLC_Metastatic_GSE136961_Hwang2020 
SU2C 


In [8]:
for(pname in names(study)){
    cat(pname,":",nrow(study[[pname]]$gs_zscore),"gene sets.\n")
}

Melanoma_Advanced_GSE91061_Riaz2017 : 169 gene sets.
Melanoma_Metastatic_GSE78220_Hugo2016 : 168 gene sets.
Melanoma_Metastatic_PRJEB23709_Gide2019 : 167 gene sets.
STAD_Metastatic_PRJEB25780_Kim2018 : 169 gene sets.
NSCLC_Advanced_GSE135222_Kim2020 : 169 gene sets.
NSCLC_Metastatic_GSE136961_Hwang2020 : 130 gene sets.
SU2C : 168 gene sets.


In [9]:
# compute null distribution of AUCs for the combined model by combining T_Inflamed with random gene sets

ntop <- median(sapply(icb_gsdb, length))
cat("Median gene set size for constructing the random gene set:", ntop,"\n")

for(one_study in names(study)){
    study[[one_study]]$rand_model <- list()

    set.seed(1357)
    study[[one_study]]$rand_model$pheno_rand <- data.frame(
        Response_Bin = study[[one_study]]$pheno$Response_Bin,
        T_Inflamed = as.numeric(study[[one_study]]$gs_zscore["T_Inflamed",]),
        randz=0, randg=0
    )
    
    med_gsize <- ntop
    study[[one_study]]$rand_model[[one_study]]$auc_randz <- c()
    study[[one_study]]$rand_model[[one_study]]$auc_randg <- c()
    
    # Null AUC is constructed by 100 random combined models
    # Random combined models are constructed in two ways: random z-scores and random gene sets
    for(i in 1:100){
        study[[one_study]]$rand_model$pheno_rand$randz <- rnorm(nrow(study[[one_study]]$rand_model$pheno_rand))
        ind_gene <- sample(1:nrow(study[[one_study]]$exp),med_gsize)
        study[[one_study]]$rand_model$pheno_rand$randg <- rowMeans(scale(t(study[[one_study]]$exp[ind_gene,])))

        test <- summary(glm(as.formula(paste0("Response_Bin ~ T_Inflamed + randz")), family = "binomial", data = study[[one_study]]$rand_model$pheno_rand))
        pred_randz <- test$coefficients["T_Inflamed",1]*study[[one_study]]$rand_model$pheno_rand$T_Inflamed+test$coefficients["randz",1]*study[[one_study]]$rand_model$pheno_rand$randz
        study[[one_study]]$rand_model$auc_randz <- c(study[[one_study]]$rand_model$auc_randz, auc(study[[one_study]]$rand_model$pheno_rand$Response_Bin, pred_randz, direction="<"))

        test <- summary(glm(as.formula(paste0("Response_Bin ~ T_Inflamed + randg")), family = "binomial", data = study[[one_study]]$rand_model$pheno_rand))
        pred_randg <- test$coefficients["T_Inflamed",1]*study[[one_study]]$rand_model$pheno_rand$T_Inflamed+test$coefficients["randg",1]*study[[one_study]]$rand_model$pheno_rand$randg
        study[[one_study]]$rand_model$auc_randg <- c(study[[one_study]]$rand_model$auc_randg, auc(study[[one_study]]$rand_model$pheno_rand$Response_Bin, pred_randg, direction="<"))
    }
}


Median gene set size for constructing the random gene set: 20 


Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control =

In [10]:
# Predict response using individual signatures

for(pname in names(study)){
    message(pname)

    res_temp <- data.frame(matrix(NA, nrow=nrow(study[[pname]]$gs_zscore), ncol=4))
    colnames(res_temp) <- c("beta","p","AIC","AUC")
    rownames(res_temp) <- rownames(study[[pname]]$gs_zscore)

    res_temp$beta <- 0
    res_temp$p <- 1

    for(the_gs in rownames(res_temp)){

        test <- summary(glm(as.formula(paste0("Response_Bin ~ ",the_gs)), family = "binomial", data = study[[pname]]$pheno))

        if(paste0(the_gs) %in% rownames(test$coefficients)){
            test_ind <- which(paste0(the_gs) == rownames(test$coefficients))
            res_temp[the_gs, "beta"] <- test$coefficients[test_ind,1]
            res_temp[the_gs, "p"] <- test$coefficients[test_ind,4]
        }
        res_temp[the_gs, "AIC"] <- test$aic

        pred <- ROCR::prediction(study[[pname]]$pheno[, paste0(the_gs)]*res_temp[the_gs,"beta"], study[[pname]]$pheno[, "Response_Bin"])
        res_temp[the_gs, "AUC"] <- ROCR::performance(pred, measure = "auc")@y.values[[1]]
    }
    
    study[[pname]]$uni_gs <- res_temp
}

Melanoma_Advanced_GSE91061_Riaz2017

Melanoma_Metastatic_GSE78220_Hugo2016

Melanoma_Metastatic_PRJEB23709_Gide2019

STAD_Metastatic_PRJEB25780_Kim2018

NSCLC_Advanced_GSE135222_Kim2020

NSCLC_Metastatic_GSE136961_Hwang2020

SU2C



In [11]:
# Figure 1A: Association between response and individual signatures

pos_sig <- study$SU2C$uni_gs[study$SU2C$uni_gs$beta > 0,]
neg_sig <- study$SU2C$uni_gs[study$SU2C$uni_gs$beta < 0,]
pos_sig <- pos_sig[order(pos_sig$p),]
neg_sig <- neg_sig[order(neg_sig$p),]

ntop <- 10

dat.plot <- rbind(pos_sig[1:ntop,], neg_sig[1:ntop,])
dat.plot$SignedLog10P <- -log10(dat.plot$p)*sign(dat.plot$beta)
dat.plot$GeneSet <- rownames(dat.plot)
dat.plot$GeneSet <- factor(dat.plot$GeneSet, levels = rev(dat.plot$GeneSet))
dat.plot$Association <- c(rep("CR/PR",ntop), rep("PD/SD",ntop))

p <- ggplot(data = dat.plot, aes(x=GeneSet, y=SignedLog10P, fill=Association)) + geom_bar(stat = "identity") +
    geom_hline(yintercept = c(log10(0.05), -log10(0.05)), linetype="dashed", color="blue") +
    coord_flip() + theme_bw() + theme(legend.position = "top") + scale_fill_brewer(palette = "Set1")
ggsave("Figure/Figure1A.pdf", p, width=6, height=5)

In [12]:
# Predict response by combining T_Inflamed with other signatures

pname <- "SU2C"

gs_vec <- rownames(study[[pname]]$gs_zscore)
gs_vec <- gs_vec[gs_vec != "T_Inflamed"]

res_temp <- data.frame(matrix(NA, nrow=length(gs_vec), ncol=10))
colnames(res_temp) <- c("v1","v2","b1","b2","p1","p2","AIC1","AIC2","AIC","AUC")

res_temp$b1 <- 0
res_temp$p1 <- 1
res_temp$b2 <- 0
res_temp$p2 <- 1

ind <- 1
for(i in 1:length(gs_vec)){
    v1 <- gs_vec[i]
    v2 <- "T_Inflamed"

    ### compute zscore

    test <- summary(glm(as.formula(paste0("Response_Bin ~ ",v1," + ",v2)), family = "binomial", data = study[[pname]]$pheno))

    res_temp[ind, "v1"] <- v1
    if(v1 %in% rownames(test$coefficients)){
        test_ind <- which(v1 == rownames(test$coefficients))
        res_temp[ind, "b1"] <- test$coefficients[test_ind,1]
        res_temp[ind, "p1"] <- test$coefficients[test_ind,4]
    }
    res_temp[ind, "v2"] <- v2
    if(v2 %in% rownames(test$coefficients)){
        test_ind <- which(v2 == rownames(test$coefficients))
        res_temp[ind, "b2"] <- test$coefficients[test_ind,1]
        res_temp[ind, "p2"] <- test$coefficients[test_ind,4]
    }
    res_temp[ind, "AIC"] <- test$aic
    res_temp[ind, "AIC1"] <- test$aic-study[[pname]]$uni_gs[v1, "AIC"]
    res_temp[ind, "AIC2"] <- test$aic-study[[pname]]$uni_gs[v2, "AIC"]

    temp_pred <- study[[pname]]$pheno[, v1]*res_temp[ind, "b1"]+study[[pname]]$pheno[, v2]*res_temp[ind, "b2"]
    pred <- ROCR::prediction(temp_pred, study[[pname]]$pheno[, "Response_Bin"])
    res_temp[ind, "AUC"] <- ROCR::performance(pred, measure = "auc")@y.values[[1]]

    ind <- ind+1
}

study[[pname]]$pair_gs <- res_temp


In [13]:
# Figure 1B: P-value changes from univariate models to combined models (with T_Inflamed) for top 10 negative signatures

pos_sig <- study$SU2C$uni_gs[study$SU2C$uni_gs$beta > 0,]
neg_sig <- study$SU2C$uni_gs[study$SU2C$uni_gs$beta < 0,]
pos_sig <- pos_sig[order(pos_sig$p),]
neg_sig <- neg_sig[order(neg_sig$p),]

ntop <- 10

selected_gs <- c(rownames(neg_sig)[1:ntop])
pivot_gs <- "T_Inflamed"

gep_pair <- study$SU2C$pair_gs[(study$SU2C$pair_gs$v1 %in% selected_gs & study$SU2C$pair_gs$v2 == "T_Inflamed"),]
gep_pair <- gep_pair[match(selected_gs, gep_pair$v1), ]

dat.plot <- data.frame(matrix(NA, ncol=5, nrow=4*ntop))
colnames(dat.plot) <- c("pval","beta","Model","Group","GeneSet")

dat.plot[1:ntop, "pval"] <- study$SU2C$uni_gs[pivot_gs,"p"]
dat.plot[1:ntop, "beta"] <- study$SU2C$uni_gs[pivot_gs,"beta"]
dat.plot[1:ntop, "Model"] <- "Univariate"
dat.plot[1:ntop, "Group"] <- paste(pivot_gs, ":", selected_gs)

dat.plot[(ntop+1):(2*ntop), "pval"] <- study$SU2C$uni_gs[selected_gs,"p"]
dat.plot[(ntop+1):(2*ntop), "beta"] <- study$SU2C$uni_gs[selected_gs,"beta"]
dat.plot[(ntop+1):(2*ntop), "Model"] <- "Univariate"
dat.plot[(ntop+1):(2*ntop), "Group"] <- selected_gs

dat.plot[(2*ntop+1):(4*ntop), "Model"] <- "Combined"

for(i in 1:nrow(gep_pair)){
    pivot_p <- gep_pair[i, "p1"]
    pivot_b <- gep_pair[i, "b1"]
    if(gep_pair[i,"v2"] == pivot_gs){
        pivot_p <- gep_pair[i, "p2"]
        pivot_b <- gep_pair[i, "b2"]
    }
    
    target_p <- gep_pair[i, "p2"]
    target_b <- gep_pair[i, "b2"]
    target_gs <- gep_pair[i, "v2"]
    if(gep_pair[i,"v1"] != pivot_gs){
        target_p <- gep_pair[i, "p1"]
        target_b <- gep_pair[i, "b1"]
        target_gs <- gep_pair[i, "v1"]
    }
    
    dat.plot[2*ntop+i, "pval"] <- pivot_p
    dat.plot[2*ntop+i, "beta"] <- pivot_b
    dat.plot[2*ntop+i, "Group"] <- paste(pivot_gs, ":", target_gs)
    
    dat.plot[3*ntop+i, "pval"] <- target_p
    dat.plot[3*ntop+i, "beta"] <- target_b
    dat.plot[3*ntop+i, "Group"] <- paste(target_gs)
    
    dat.plot[i, "GeneSet"] <- pivot_gs
    dat.plot[ntop + i, "GeneSet"] <- "Top10_Negative"
    dat.plot[2*ntop + i, "GeneSet"] <- pivot_gs
    dat.plot[3*ntop + i, "GeneSet"] <- "Top10_Negative"
}

dat.plot$Model <- factor(dat.plot$Model, levels=c("Univariate","Combined"))
dat.plot$SignedLog10P <- -log10(dat.plot$pval)*sign(dat.plot$beta)
p <- ggplot(data = dat.plot, aes(x=Model, y=SignedLog10P, group=Group)) + geom_line() + geom_point(aes(color=GeneSet), size=2) +
    geom_hline(yintercept = c(-log10(0.05), 0, log10(0.05)), linetype=c("dashed","solid","dashed"), color=c("blue","darkgrey","blue")) +
    theme_bw() + scale_color_brewer(palette = "Set1")
ggsave("Figure/Figure1B.pdf", p, width=6, height=4)



# # Figure 1C: AIC changes from univariate models to combined models (with T_Inflamed) for top 10 negative signatures

temp_t <- study$SU2C$pair_gs[study$SU2C$pair_gs$v2 == "T_Inflamed",]
dat.plot <- data.frame(
    Model = c(rep("Univariate", 11), rep("Combined with\nT_Inflamed", 10)),
    AIC = c(study$SU2C$uni_gs[c("T_Inflamed", selected_gs),"AIC"], temp_t[match(selected_gs, temp_t$v1), "AIC"]),
    GeneSet = c("T_Inflamed",selected_gs, selected_gs)
)

dat.plot$Model <- factor(dat.plot$Model, levels=c("Univariate","Combined with\nT_Inflamed"))

p <- ggplot(data = dat.plot, aes(x=Model, y=AIC, group=GeneSet, color=GeneSet)) + geom_point() + geom_line() + theme_bw()
ggsave("Figure/Figure1C.pdf", p, width=6, height=3)

In [14]:
# Figure 1D: AUC changes from univariate models to combined models (with T_Inflamed) for top 10 negative signatures

pos_sig <- study$SU2C$uni_gs[study$SU2C$uni_gs$beta > 0,]
neg_sig <- study$SU2C$uni_gs[study$SU2C$uni_gs$beta < 0,]
pos_sig <- pos_sig[order(pos_sig$p),]
neg_sig <- neg_sig[order(neg_sig$p),]

ntop <- 10

selected_gs <- c(rownames(neg_sig)[1:ntop])

dat.plot <- data.frame(
    GeneSet = c(paste0("T_Inflamed+", selected_gs), "T_Inflamed+randg at p=0.05", "T_Inflamed"),
    AUC = 0,
    Type = c(rep("Real", length(selected_gs)), "Random","Real")
)
rownames(dat.plot) <- c(selected_gs, "randg", "T_Inflamed")
dat.plot$GeneSet <- factor(dat.plot$GeneSet, levels = dat.plot$GeneSet)

dat.plot$AUC[which(dat.plot$GeneSet == "T_Inflamed")] <- study$SU2C$uni_gs["T_Inflamed", "AUC"]
dat.plot$AUC[which(dat.plot$GeneSet == "T_Inflamed+randg at p=0.05")] <- sort(study$SU2C$rand_model$auc_randg)[95]

dat.plot[selected_gs, "AUC"] <- study$SU2C$pair_gs[match(selected_gs, study$SU2C$pair_gs$v1), "AUC"]


p <- ggplot(data = dat.plot, aes(x=GeneSet, y=AUC, fill=Type)) + geom_bar(stat="identity") +
    geom_hline(yintercept = c(dat.plot$AUC[nrow(dat.plot)], dat.plot["randg","AUC"]), linetype="dotted", color=c("blue","red")) +
    scale_fill_manual(values = c("grey","#505050")) + 
    coord_flip(ylim=c(0.5,0.7)) + theme_bw() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), legend.position = "none")
ggsave("Figure/Figure1D.pdf", p, width=6, height=4)

In [15]:
# Select myeloid-based signatures that significantly better predicted response when combining with T_Inflamed

pname <- "SU2C"

pair_filter <- study[[pname]]$pair_gs$v2 == "T_Inflamed" & study[[pname]]$pair_gs$b1 < 0 & study[[pname]]$pair_gs$b2 > 0 &
    study[[pname]]$pair_gs$p1 < 0.05 & study[[pname]]$pair_gs$p2 < 0.05 &
    study[[pname]]$pair_gs$p1 < study[[pname]]$uni_gs[study[[pname]]$pair_gs$v1,"p"] &
    study[[pname]]$pair_gs$p2 < study[[pname]]$uni_gs[study[[pname]]$pair_gs$v2,"p"] &
    study[[pname]]$pair_gs$AUC > sort(study[[pname]]$rand_model$auc_randg)[95]

study[[pname]]$pair_neg <- list()
study[[pname]]$pair_neg$stat <- study[[pname]]$pair_gs[pair_filter,]
study[[pname]]$pair_neg$gs <- study[[pname]]$pair_gs[pair_filter, "v1"]


In [16]:
# SFigure 1

pname <- "SU2C"

dat.plot <- data.frame(matrix(0, nrow = length(study[[pname]]$pair_neg$gs), ncol=2))
rownames(dat.plot) <- study[[pname]]$pair_neg$gs
colnames(dat.plot) <- c("T_Inflamed","Myeloid")

lr.t <- glm(as.formula(paste0("Response_Bin ~ T_Inflamed")), family = "binomial", data = study[[pname]]$pheno)
for(one_gs in study[[pname]]$pair_neg$gs){
    lr.uni <- glm(as.formula(paste0("Response_Bin ~ ", one_gs)), family = "binomial", data = study[[pname]]$pheno)
    lr.comb <- glm(as.formula(paste0("Response_Bin ~ T_Inflamed + ", one_gs)), family = "binomial", data = study[[pname]]$pheno)
    dat.plot[one_gs, "T_Inflamed"] <- anova(lr.t,lr.comb,test="LRT")[2,"Pr(>Chi)"]
    dat.plot[one_gs, "Myeloid"] <- anova(lr.uni,lr.comb,test="LRT")[2,"Pr(>Chi)"]
}

dat.plot <- -log10(dat.plot)
dat.plot$Model <- paste0("Response ~ T_Inflamed + ", study[[pname]]$pair_neg$gs)


p1 <- ggplot(data = dat.plot, aes(x = Model, y = T_Inflamed)) + geom_bar(stat = "identity") + geom_hline(yintercept = -log10(0.05), color = "blue", linetype="dashed") +
    labs(title = "Response ~ T_Inflamed", y = "-Log10P (LRT test)") + coord_flip() + theme_bw()
p2 <- ggplot(data = dat.plot, aes(x = Model, y = Myeloid)) + geom_bar(stat = "identity") + geom_hline(yintercept = -log10(0.05), color = "blue", linetype="dashed") +
    labs(title = "Response ~ NegSignature", y = "-Log10P (LRT test)") + coord_flip() + theme_bw() + theme(axis.text.y = element_blank(), axis.title.y = element_blank())
ggsave("Figure/SFigure1.pdf", p1+p2, width = 10, height = 6)

In [17]:
# Load NSCLC scRNA-seq data from Salcher et al. (2022)

library(Seurat)

sobj <- readRDS("~/datasets/scRNA/references/lung/Neutrophils_NSCLC_Salcher2022/core_symbol.rds")
sobj <- subset(sobj, cells = colnames(sobj)[sobj@meta.data$disease %in% c("non-small cell lung carcinoma",
                                                                          "squamous cell lung carcinoma","lung adenocarcinoma")])

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [18]:
# Figure 2: expression of myeloid-based signatures across NSCLC cell types

pname <- "SU2C"

show_gs <- c()

for(one_gs in c("T_Inflamed", study[[pname]]$pair_neg$gs)){
    valid_genes <- intersect(rownames(sobj[["RNA"]]), icb_gsdb[[one_gs]])
    if(length(valid_genes) == 0){
        next
    }

    if(length(valid_genes) == 1){
        sobj@meta.data[[one_gs]] <- as.numeric(sobj@assays$RNA@data[valid_genes,])
    } else {
        sobj@meta.data[[one_gs]] <- colMeans(sobj@assays$RNA@data[valid_genes,])
    }

    show_gs <- c(show_gs, one_gs)
}

gs_mtx <- data.frame(matrix(NA, nrow=length(unique(sobj@meta.data$cell_type)), ncol=length(show_gs)))
rownames(gs_mtx) <- unique(sobj@meta.data$cell_type)
colnames(gs_mtx) <- show_gs

for(ctype in rownames(gs_mtx)){
    gs_mtx[ctype, ] <- colMeans(sobj@meta.data[sobj@meta.data$cell_type == ctype, show_gs])
}

col.anno <- data.frame(
    nLog10P = c(NA, -log10(study[[pname]]$pair_neg$stat[match(show_gs[2:length(show_gs)], study[[pname]]$pair_neg$stat$v1), "p1"])),
    AUC = c(NA, study[[pname]]$pair_neg$stat[match(show_gs[2:length(show_gs)], study[[pname]]$pair_neg$stat$v1), "AUC"])
)
rownames(col.anno) <- show_gs

temp <- scale(gs_mtx)
temp[temp < -2] <- -2
temp[temp > 2] <- 2

pheatmap::pheatmap(temp, breaks = seq(-2,2,4/100), color = colorRampPalette(rev(RColorBrewer::brewer.pal(n = 7, name = "RdBu")))(100),
         filename="Figure/Figure2.pdf", width=11, height=9)

In [19]:
# SFigure 3: Deconvolution TIMER

deconv_est <- read.csv("Data/TIMER_est.csv")
rownames(deconv_est) <- deconv_est[,1]
deconv_est <- deconv_est[,-1]

deconv_est <- deconv_est[, rownames(study$SU2C$pheno)]

cor_mat <- data.frame(matrix(0, nrow = length(study$SU2C$pair_neg$gs), ncol = nrow(deconv_est)))
rownames(cor_mat) <- study$SU2C$pair_neg$gs
colnames(cor_mat) <- rownames(deconv_est)

for(one_gs in rownames(cor_mat)){
    for(one_ct in colnames(cor_mat)){
        cor_mat[one_gs, one_ct] <- cor(study$SU2C$pheno[[one_gs]], as.numeric(deconv_est[one_ct,]))
    }
}

pheatmap::pheatmap(cor_mat, breaks = seq(-0.5,0.5,1/100), color = colorRampPalette(rev(RColorBrewer::brewer.pal(n = 7, name = "RdBu")))(100),
         filename="Figure/SFigure3_TIMER.pdf", width=6, height=6)

In [20]:
# SFigure 3: Deconvolution iSort

deconv_est <- read.csv("Data/iSort_out.csv", row.names=1)

deconv_est <- deconv_est[rownames(study$SU2C$pheno),]

cor_mat <- data.frame(matrix(0, nrow = length(study$SU2C$pair_neg$gs), ncol = ncol(deconv_est)))
rownames(cor_mat) <- study$SU2C$pair_neg$gs
colnames(cor_mat) <- colnames(deconv_est)

for(one_gs in rownames(cor_mat)){
    for(one_ct in colnames(cor_mat)){
        cor_mat[one_gs, one_ct] <- cor(study$SU2C$pheno[[one_gs]], as.numeric(deconv_est[,one_ct]))
    }
}

pheatmap::pheatmap(cor_mat, breaks = seq(-0.5,0.5,1/100), color = colorRampPalette(rev(RColorBrewer::brewer.pal(n = 7, name = "RdBu")))(100),
         filename="Figure/SFigure3_iSort.pdf", width=10, height=8)

In [21]:
# Cross-validation

frac <- 0.5

better_auc <- data.frame(matrix(0, nrow = length(study$SU2C$pair_neg$gs), ncol = 2))
rownames(better_auc) <- study$SU2C$pair_neg$gs
colnames(better_auc) <- c("T_Inflamed","NegSignature")

set.seed(1357)

for(i in 1:100){

    train_ind <- sample(1:nrow(study$SU2C$pheno), as.integer(nrow(study$SU2C$pheno)*frac))
    test_ind <- setdiff(1:nrow(study$SU2C$pheno), train_ind)

    for(one_gs in study$SU2C$pair_neg$gs){
        fit.t <- summary(glm(as.formula(paste0("Response_Bin ~ T_Inflamed")), family = "binomial", data = study[[pname]]$pheno[train_ind,]))
        fit.n <- summary(glm(as.formula(paste0("Response_Bin ~ ", one_gs)), family = "binomial", data = study[[pname]]$pheno[train_ind,]))
        fit.comb <- summary(glm(as.formula(paste0("Response_Bin ~ T_Inflamed + ", one_gs)), family = "binomial", data = study[[pname]]$pheno[train_ind,]))

        b_t_alone <- fit.t$coefficients[2,1]
        b_n_alone <- fit.n$coefficients[2,1]
        b_t_comb <- fit.comb$coefficients[2,1]
        b_n_comb <- fit.comb$coefficients[3,1]
        
#         auc_t_alone <- auc(study$SU2C$pheno[test_ind, "Response_Bin"], study$SU2C$pheno[["T_Inflamed"]][test_ind]*b_t_alone, direction="<")
#         auc_n_alone <- auc(study$SU2C$pheno[test_ind, "Response_Bin"], study$SU2C$pheno[[one_gs]][test_ind]*b_n_alone, direction="<")
#         auc_comb <- auc(study$SU2C$pheno[test_ind, "Response_Bin"], study$SU2C$pheno[[one_gs]][test_ind]*b_n_comb+study$SU2C$pheno[["T_Inflamed"]][test_ind]*b_t_comb, direction="<")

        pred <- ROCR::prediction(study$SU2C$pheno[["T_Inflamed"]][test_ind]*b_t_alone, study$SU2C$pheno[test_ind, "Response_Bin"])
        auc_t_alone <- ROCR::performance(pred, measure = "auc")@y.values[[1]]
        pred <- ROCR::prediction(study$SU2C$pheno[[one_gs]][test_ind]*b_n_alone, study$SU2C$pheno[test_ind, "Response_Bin"])
        auc_n_alone <- ROCR::performance(pred, measure = "auc")@y.values[[1]]
        pred <- ROCR::prediction(study$SU2C$pheno[[one_gs]][test_ind]*b_n_comb+study$SU2C$pheno[["T_Inflamed"]][test_ind]*b_t_comb, study$SU2C$pheno[test_ind, "Response_Bin"])
        auc_comb <- ROCR::performance(pred, measure = "auc")@y.values[[1]]

        if(auc_comb > auc_t_alone) better_auc[one_gs, "T_Inflamed"] <- better_auc[one_gs, "T_Inflamed"]+1
        if(auc_comb > auc_n_alone) better_auc[one_gs, "NegSignature"] <- better_auc[one_gs, "NegSignature"]+1
    }
}

In [22]:
sum(apply(better_auc >= 75, 1, all))
cat("Least significant p-value:",pbinom(q = min(better_auc[,c("T_Inflamed","NegSignature")]), size = 100, prob = 0.5, lower.tail = F),"\n")

[1] 23

Least significant p-value: 9.050013e-08 


In [23]:
# SFigure 2: plot cross-validation

better_auc$Model <- paste0("T_Inflamed+", rownames(better_auc))

p1 <- ggplot(data = better_auc, aes(x = Model, y = T_Inflamed)) + geom_bar(stat = "identity") +
    geom_hline(yintercept = qbinom(p = 0.05, size = 100, prob = 0.5, lower.tail = F), color = "blue", linetype="dashed") +
    labs(title = "Compare with T_Inflamed alone", y = "Frequency of weighted sum with higher AUC", x = "Model for computing weighted sum") + coord_flip() + theme_bw()
p2 <- ggplot(data = better_auc, aes(x = Model, y = NegSignature)) + geom_bar(stat = "identity") + 
    geom_hline(yintercept = qbinom(p = 0.05, size = 100, prob = 0.5, lower.tail = F), color = "blue", linetype="dashed") +
    labs(title = "Compare with negative signature alone", y = "Frequency of weighted sum with higher AUC", x = "Model for computing weighted sum") + coord_flip() + theme_bw() +
    theme(axis.text.y = element_blank(), axis.title.y = element_blank())
ggsave("Figure/SFigure2.pdf", p1+p2, width = 10, height = 6)

In [24]:
# Figure 3

library(ComplexHeatmap)

pname <- "SU2C"

temp_rank <- data.frame(matrix(NA, ncol=length(study[[pname]]$pair_neg$gs), nrow=nrow(study[[pname]]$pheno)))
colnames(temp_rank) <- paste0("T+",study[[pname]]$pair_neg$gs)
rownames(temp_rank) <- rownames(study[[pname]]$pheno)

for(one_gs in study[[pname]]$pair_neg$gs){
    ind_gs <- which(study[[pname]]$pair_neg$stat$v1 == one_gs)
    pred_comb <- study[[pname]]$pair_neg$stat[ind_gs, "b1"]*study[[pname]]$pheno[[one_gs]]+
        study[[pname]]$pair_neg$stat[ind_gs, "b2"]*study[[pname]]$pheno$T_Inflamed
    temp_rank[[paste0("T+",one_gs)]] <- rank(-pred_comb)-rank(-study[[pname]]$pheno$T_Inflamed)
}

temp_rank <- t(temp_rank)

col.anno <- data.frame(
    T_Inflamed_rank = rank(study[[pname]]$pheno$T_Inflamed), # rank from small to large to match the gradient coloring from white to dark red
    Response = study[[pname]]$pheno$Response_Bin,
    Immune_class = study[[pname]]$pheno$M_cluster
)
rownames(col.anno) <- colnames(temp_rank)


set.seed(138)

p1w <- ComplexHeatmap::pheatmap(temp_rank, color = colorRampPalette(RColorBrewer::brewer.pal(n = 7, name = "PRGn"))(100), show_colnames = F,
                               annotation_legend = T, annotation_col = col.anno,
                               border_color = "grey60", name="Rank_Diff", row_names_max_width = unit(13, "cm"),
                               fontsize = 8, treeheight_row = 5, treeheight_col = 5,
                              heatmap_legend_param = list(legend_direction = "horizontal", legend_width = unit(3, "cm"), legend_gp = gpar(fontsize = 8)))

plot_gs <- c("Salcher2022_non_classical_monocyte","Salcher2022_neutrophil","Bagaev2021_Matrix_remodeling","Danaher2017_Neutrophil",
             "T_Inflamed")

exp_mtx <- t(study[[pname]]$pheno[,plot_gs])
exp_mtx[exp_mtx>1] <- 1
exp_mtx[exp_mtx<-1] <- -1

p2 <- ComplexHeatmap::pheatmap(exp_mtx, breaks = seq(-1,1,2/100), annotation_legend=F,
                               color = colorRampPalette(rev(RColorBrewer::brewer.pal(n = 7, name = "RdBu")))(100), 
                               show_colnames = F, , border_color = "grey60", name="Exp_Z", row_names_max_width = unit(13, "cm"),
                               fontsize = 8, treeheight_row = 5,
                               heatmap_legend_param = list(legend_direction = "horizontal", legend_width = unit(3, "cm"), legend_gp = gpar(fontsize = 8)))


pdf(file="Figure/Figure3.pdf", width = 12, height=7)
ComplexHeatmap::draw(p1w %v% p2, heatmap_legend_side="bottom", annotation_legend_side="bottom")
dev.off()

ComplexHeatmap version 2.18.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))
! pheatmap() has been masked by ComplexHeatmap::pheatmap(). Most of the arguments
   in the original pheatmap() are identically supported in the new function. You 
   can still use the original function by explicitly calling pheatmap::pheatmap().



Attaching package: ‘ComplexHeatmap’


The following ob

png 
  2

In [25]:
# Figure 4 and SFigure 1: survival plots (old)

pname <- "SU2C"

for(one_gs in study[[pname]]$pair_neg$gs){
    ind_gs <- which(study[[pname]]$pair_neg$stat$v1 == one_gs)
    study[[pname]]$pheno[[paste0("T_", one_gs)]] <- study[[pname]]$pair_neg$stat[ind_gs, "b1"]*study[[pname]]$pheno[[one_gs]]+
        study[[pname]]$pair_neg$stat[ind_gs, "b2"]*study[[pname]]$pheno$T_Inflamed
}

univ_formulas <- sapply(c("T_Inflamed", paste0("T_",study[[pname]]$pair_neg$gs)),
                        function(x){as.formula(paste('Surv(Harmonized_OS_Months, Harmonized_OS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_model(model_list = univ_models,  merge_models =T)
ggsave("Figure/Figure4A_old.pdf", p, width=10, height=8)


univ_formulas <- sapply(c("T_Inflamed", study[[pname]]$pair_neg$gs),
                        function(x){as.formula(paste('Surv(Harmonized_OS_Months, Harmonized_OS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_model(model_list = univ_models,  merge_models =T)

ggsave("Figure/SFigure1A_old.pdf", p, width=10, height=8)

univ_formulas <- sapply(c("T_Inflamed", paste0("T_",study[[pname]]$pair_neg$gs)),
                        function(x){as.formula(paste('Surv(Harmonized_PFS_Months, Harmonized_PFS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_model(model_list = univ_models,  merge_models =T)
ggsave("Figure/Figure4B_old.pdf", p, width=10, height=8)

univ_formulas <- sapply(c("T_Inflamed", study[[pname]]$pair_neg$gs),
                        function(x){as.formula(paste('Surv(Harmonized_PFS_Months, Harmonized_PFS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_model(model_list = univ_models,  merge_models =T)

ggsave("Figure/SFigure1B_old.pdf", p, width=10, height=8)

Warning message in recalculate_width_panels(panel_positions, mapped_text = mapped_text, :
“Unable to resize forest panel to be smaller than its heading; consider a smaller text size”
Warning message in recalculate_width_panels(panel_positions, mapped_text = mapped_text, :
“Unable to resize forest panel to be smaller than its heading; consider a smaller text size”
Warning message in recalculate_width_panels(panel_positions, mapped_text = mapped_text, :
“Unable to resize forest panel to be smaller than its heading; consider a smaller text size”
Warning message in recalculate_width_panels(panel_positions, mapped_text = mapped_text, :
“Unable to resize forest panel to be smaller than its heading; consider a smaller text size”


In [26]:
# Functions for forest plot

get_scale <- function(plot, width_wanted, height_wanted, unit = "in"){
  h <- convertHeight(sum(plot$heights), unit, TRUE)
  w <- convertWidth(sum(plot$widths), unit, TRUE)
  max(c(w/width_wanted,  h/height_wanted))
}

forest_plot <- function(model_list, model_names, xlim, ticks_at){
    dt <- data.frame(
        Model = model_names,
        est = sapply(model_names, function(x){summary(model_list[[x]])$conf.int[1,1]}),
        low = sapply(model_names, function(x){summary(model_list[[x]])$conf.int[1,3]}),
        hi = sapply(model_names, function(x){summary(model_list[[x]])$conf.int[1,4]}),
        P = as.character(round(sapply(model_names, function(x){summary(model_list[[x]])$coefficients[1,5]}),3)),
        HR = paste(rep(" ", 10), collapse = " "),
        C_index = as.character(round(sapply(model_names, function(x){model_list[[x]]$concordance["concordance"]}),3))
    )

    dt$`HR (95% CI)` <- paste0(round(dt$est,2)," (",round(dt$low,2)," - ",round(dt$hi,2),")")
    dt$P <- sapply(dt$P, function(x){if(x == "0") "<0.001" else x})

    p <- forest(
        dt[,c("Model","HR","HR (95% CI)","P","C_index")],
        est = list(dt$est), lower = list(dt$low), upper = list(dt$hi),
        ci_column = 2, xlim = xlim, ticks_at = ticks_at, ref_line = 1,
        arrow_lab = c("Better", "Worse")
    )
    return(p)
}

In [27]:
# Figure 4 and SFigure 4: survival plots

pname <- "SU2C"

for(one_gs in study[[pname]]$pair_neg$gs){
    ind_gs <- which(study[[pname]]$pair_neg$stat$v1 == one_gs)
    study[[pname]]$pheno[[paste0("T_", one_gs)]] <- study[[pname]]$pair_neg$stat[ind_gs, "b1"]*study[[pname]]$pheno[[one_gs]]+
        study[[pname]]$pair_neg$stat[ind_gs, "b2"]*study[[pname]]$pheno$T_Inflamed
}

univ_formulas <- sapply(c("T_Inflamed", paste0("T_",study[[pname]]$pair_neg$gs)),
                        function(x){as.formula(paste('Surv(Harmonized_OS_Months, Harmonized_OS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})

p <- forest_plot(model_list = univ_models, model_names = c("T_Inflamed", paste0("T_",study[[pname]]$pair_neg$gs)), xlim = c(0, 1.5), ticks_at = c(0.2, 1, 1.5))
                   
p_sc <- get_scale(plot = p, width_wanted = 10, height_wanted = 8, unit = "in")
ggsave(filename = "Figure/Figure4A.pdf", plot = p, width = 10, height = 8, scale = p_sc)
                   
univ_formulas <- sapply(c("T_Inflamed", study[[pname]]$pair_neg$gs),
                        function(x){as.formula(paste('Surv(Harmonized_OS_Months, Harmonized_OS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_plot(model_list = univ_models, model_names = c("T_Inflamed", study[[pname]]$pair_neg$gs), xlim = c(0, 2), ticks_at = c(0.5, 1, 1.5, 2))

p_sc <- get_scale(plot = p, width_wanted = 10, height_wanted = 8, unit = "in")
ggsave("Figure/SFigure4A.pdf", p, width=10, height=8, scale = p_sc)

univ_formulas <- sapply(c("T_Inflamed", paste0("T_",study[[pname]]$pair_neg$gs)),
                        function(x){as.formula(paste('Surv(Harmonized_PFS_Months, Harmonized_PFS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_plot(model_list = univ_models, model_names = c("T_Inflamed", paste0("T_",study[[pname]]$pair_neg$gs)), xlim = c(0, 1.5), ticks_at = c(0.2, 1, 1.5))
p_sc <- get_scale(plot = p, width_wanted = 10, height_wanted = 8, unit = "in")
ggsave("Figure/Figure4B.pdf", p, width=10, height=8, scale = p_sc)

univ_formulas <- sapply(c("T_Inflamed", study[[pname]]$pair_neg$gs),
                        function(x){as.formula(paste('Surv(Harmonized_PFS_Months, Harmonized_PFS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_plot(model_list = univ_models, model_names = c("T_Inflamed", study[[pname]]$pair_neg$gs), xlim = c(0, 2), ticks_at = c(0.5, 1, 1.5, 2))
p_sc <- get_scale(plot = p, width_wanted = 10, height_wanted = 8, unit = "in")
ggsave("Figure/SFigure4B.pdf", p, width=10, height=8, scale = p_sc)

In [70]:
# Generate Stabl input data for feature selection

write.csv(data.frame(sid=rownames(study[[pname]]$pheno), Response_Bin=study[[pname]]$pheno$Response_Bin), "Data/Stabl/response.csv", row.names=F)

neg_genes <- intersect(unique(unlist(icb_gsdb[study[[pname]]$pair_neg$gs])), rownames(study[[pname]]$exp))

stabl_in <- cbind(
    sid=rownames(study[[pname]]$pheno),
    T_Inflamed=study[[pname]]$pheno$T_Inflamed,
    t(study[[pname]]$exp[neg_genes, rownames(study[[pname]]$pheno)])
)
write.csv(stabl_in, "Data/Stabl/T_Inflamed_paired_neg_genes.csv", row.names=F)

In [28]:
# Construct immune-altered signatures

pname <- "SU2C"

study[[pname]]$stabl <- list()

# Read Stabl output

stabl_out <- cbind(read.csv("Data/Stabl/Max STABL scores.csv", row.names=1), beta=0, pval=1)
stabl_out <- stabl_out[-which(rownames(stabl_out) == "T_Inflamed"),]

temp_pheno <- cbind(study[[pname]]$pheno[,c("Response_Bin","T_Inflamed")], scale(t(study[[pname]]$exp[rownames(stabl_out),rownames(study[[pname]]$pheno)])))

for(i in 1:nrow(stabl_out)){
    one_gene <- rownames(stabl_out)[i]
    if(grepl("\\-", rownames(stabl_out)[i])){
        one_gene <- gsub("\\-","_",rownames(stabl_out)[i])
        colnames(temp_pheno)[i+2] <- one_gene
    }
    test <- summary(glm(as.formula(paste0("Response_Bin ~ ",one_gene)), family = "binomial", data = temp_pheno))
    if(one_gene %in% rownames(test$coefficients)){
        test_ind <- which(one_gene == rownames(test$coefficients))
        stabl_out[i, "beta"] <- test$coefficients[test_ind,1]
        stabl_out[i, "pval"] <- test$coefficients[test_ind,4]
    }
}

# select Stabl-ranked features that were negatively associated with response

study[[pname]]$stabl$stabl_out <- stabl_out
study[[pname]]$stabl$neg_stat <- stabl_out[stabl_out$beta < 0,]

# Predict response by 5 immune-altered signatures alone or combined with T_Inflamed

topn_range <- c(10,15,20,25,30)

topn_uni <- data.frame(matrix(NA, ncol=4, nrow=length(topn_range)))
rownames(topn_uni) <- paste0("Stabl_top",topn_range)
colnames(topn_uni) <- c("beta","p","AIC","AUC")
topn_uni$beta <- 0
topn_uni$p <- 1

topn_pair <- data.frame(matrix(NA, ncol=6, nrow=length(topn_range)))
rownames(topn_pair) <- paste0("Stabl_top",topn_range)
colnames(topn_pair) <- c("b1","b2","p1","p2","AIC","AUC")
topn_pair$b1 <- 0
topn_pair$b2 <- 0
topn_pair$p1 <- 1
topn_pair$p2 <- 1

for(topn in topn_range){
    study[[pname]]$stabl[[paste0("Stabl_top",topn)]] <- rownames(study[[pname]]$stabl$neg_stat)[1:topn]
    study[[pname]]$pheno[[paste0("Stabl_top",topn)]] <- rowMeans(scale(t(study[[pname]]$exp[study[[pname]]$stabl[[paste0("Stabl_top",topn)]],])))

    test <- summary(glm(as.formula(paste0("Response_Bin ~ Stabl_top",topn)), family = "binomial", data = study[[pname]]$pheno))
    if(paste0("Stabl_top",topn) %in% rownames(test$coefficients)){
        test_ind <- which(paste0("Stabl_top",topn) == rownames(test$coefficients))
        topn_uni[paste0("Stabl_top",topn), "beta"] <- test$coefficients[test_ind,1]
        topn_uni[paste0("Stabl_top",topn), "p"] <- test$coefficients[test_ind,4]
    }
    topn_uni[paste0("Stabl_top",topn), "AIC"] <- test$aic
    topn_uni[paste0("Stabl_top",topn), "AUC"] <- auc(study[[pname]]$pheno$Response_Bin, -study[[pname]]$pheno[[paste0("Stabl_top",topn)]], direction="<")

    test <- summary(glm(as.formula(paste0("Response_Bin ~ T_Inflamed + Stabl_top",topn)), family = "binomial", data = study[[pname]]$pheno))
    if(paste0("T_Inflamed") %in% rownames(test$coefficients)){
        test_ind <- which(paste0("T_Inflamed") == rownames(test$coefficients))
        topn_pair[paste0("Stabl_top",topn), "b1"] <- test$coefficients[test_ind,1]
        topn_pair[paste0("Stabl_top",topn), "p1"] <- test$coefficients[test_ind,4]
    }
    if(paste0("Stabl_top",topn) %in% rownames(test$coefficients)){
        test_ind <- which(paste0("Stabl_top",topn) == rownames(test$coefficients))
        topn_pair[paste0("Stabl_top",topn), "b2"] <- test$coefficients[test_ind,1]
        topn_pair[paste0("Stabl_top",topn), "p2"] <- test$coefficients[test_ind,4]
    }
    topn_pair[paste0("Stabl_top",topn), "AIC"] <- test$aic
    study[[pname]]$pheno[[paste0("T_Inflamed_Stabl_top",topn)]] <- study[[pname]]$pheno$T_Inflamed*topn_pair[paste0("Stabl_top",topn), "b1"] +
        study[[pname]]$pheno[[paste0("Stabl_top",topn)]]*topn_pair[paste0("Stabl_top",topn), "b2"]

    topn_pair[paste0("Stabl_top",topn), "AUC"] <- auc(study[[pname]]$pheno$Response_Bin, study[[pname]]$pheno[[paste0("T_Inflamed_Stabl_top",topn)]], direction="<")
}

study[[pname]]$stabl$topn_uni <- topn_uni
study[[pname]]$stabl$topn_pair <- topn_pair

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1



In [29]:
# Figure 5A: expression of Stabl-selected genes in NSCLC cell types

selected_genes <- study$SU2C$stabl[[paste0("Stabl_top30")]]
Idents(sobj) <- "cell_type"
p <- DotPlot(sobj, features = selected_genes) + RotatedAxis() + theme_bw() + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
ggsave("Figure/Figure5A.pdf", p, width=9, height=7)

Warning message:
“The following requested variables were not found: SNORA81, NCF1B”


In [30]:
# Figures 5B and 5C: AIC and AUC for immune-altered signatures alone and when combining with T_Inflamed

pname <- "SU2C"

topn_range <- c(10,15,20,25,30)

dat.plot <- data.frame(
    Model = c(rep("Univariate", length(topn_range)+1), rep("Combined with\nT_Inflamed", length(topn_range))),
    AIC = c(study[[pname]]$uni_gs["T_Inflamed","AIC"], study[[pname]]$stabl$topn_uni$AIC, study[[pname]]$stabl$topn_pair$AIC),
    AUC = c(study[[pname]]$uni_gs["T_Inflamed","AUC"], study[[pname]]$stabl$topn_uni$AUC, study[[pname]]$stabl$topn_pair$AUC),
    TopGene = c("T_Inflamed", paste0("Top",topn_range), paste0("Top",topn_range))
)
dat.plot$Model <- factor(dat.plot$Model, levels = c("Univariate","Combined with\nT_Inflamed"))

p <- ggplot(data = dat.plot, aes(x=Model, y=AIC, group=TopGene, color=TopGene)) + geom_point() + geom_line() + theme_bw()
ggsave("Figure/Figure5B.pdf", p, width=4, height=3)

p <- ggplot(data = dat.plot, aes(x=Model, y=AUC, group=TopGene, color=TopGene)) + geom_point() + geom_line() +
    geom_hline(yintercept = sort(study[[pname]]$rand_model$auc_randg)[95], linetype="dashed", color="darkgrey") + theme_bw()
ggsave("Figure/Figure5C.pdf", p, width=4, height=3)

In [31]:
# SFigure 5

pname <- "SU2C"

topn_range <- c(10,15,20,25,30)

dat.plot <- data.frame(matrix(0, nrow = length(topn_range), ncol=2))
rownames(dat.plot) <- paste0("Stabl_top",topn_range)
colnames(dat.plot) <- c("T_Inflamed","Immune_altered")

lr.t <- glm(as.formula(paste0("Response_Bin ~ T_Inflamed")), family = "binomial", data = study[[pname]]$pheno)
for(one_gs in paste0("Stabl_top",topn_range)){
    lr.uni <- glm(as.formula(paste0("Response_Bin ~ ", one_gs)), family = "binomial", data = study[[pname]]$pheno)
    lr.comb <- glm(as.formula(paste0("Response_Bin ~ T_Inflamed + ", one_gs)), family = "binomial", data = study[[pname]]$pheno)
    dat.plot[one_gs, "T_Inflamed"] <- anova(lr.t,lr.comb,test="LRT")[2,"Pr(>Chi)"]
    dat.plot[one_gs, "Immune_altered"] <- anova(lr.uni,lr.comb,test="LRT")[2,"Pr(>Chi)"]
}

dat.plot <- -log10(dat.plot)
dat.plot$Model <- paste0("Response ~ T_Inflamed + ", paste0("Stabl_top",topn_range))


p1 <- ggplot(data = dat.plot, aes(x = Model, y = T_Inflamed)) + geom_bar(stat = "identity") + geom_hline(yintercept = -log10(0.05), color = "blue", linetype="dashed") +
    labs(title = "Response ~ T_Inflamed", y = "-Log10P (LRT test)") + coord_flip() + theme_bw()
p2 <- ggplot(data = dat.plot, aes(x = Model, y = Immune_altered)) + geom_bar(stat = "identity") + geom_hline(yintercept = -log10(0.05), color = "blue", linetype="dashed") +
    labs(title = "Response ~ immune-altered signature", y = "-Log10P (LRT test)") + coord_flip() + theme_bw() + theme(axis.text.y = element_blank(), axis.title.y = element_blank())
ggsave("Figure/SFigure5.pdf", p1+p2, width = 10, height = 4)

In [32]:
# SFigure 6: predict OS and PFS using immune-altered signatures alone or combining with T_Inflamed

pname <- "SU2C"

topn_range <- c(10,15,20,25,30)

univ_formulas <- sapply(c("T_Inflamed", paste0("Stabl_top",topn_range), paste0("T_Inflamed_Stabl_top",topn_range)),
                        function(x){as.formula(paste('Surv(Harmonized_OS_Months, Harmonized_OS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})
p <- forest_plot(model_list = univ_models, model_names = c("T_Inflamed", paste0("Stabl_top",topn_range), paste0("T_Inflamed_Stabl_top",topn_range)),
                 xlim = c(0, 3.5), ticks_at = c(0.2, 1, 1.5, 2, 2.5, 3.5))
                   
p_sc <- get_scale(plot = p, width_wanted = 10, height_wanted = 8, unit = "in")
ggsave(filename = "Figure/SFigure6A.pdf", plot = p, width = 10, height = 8, scale = p_sc)

# p <- forest_model(model_list = univ_models,  merge_models =T)
# ggsave("Figure/SFigure6A_old.pdf", p, width=8, height=8)

univ_formulas <- sapply(c("T_Inflamed", paste0("Stabl_top",topn_range), paste0("T_Inflamed_Stabl_top",topn_range)),
                        function(x){as.formula(paste('Surv(Harmonized_PFS_Months, Harmonized_PFS_Event)~', x))})
univ_models <- lapply(univ_formulas, function(x){coxph(x, data = study[[pname]]$pheno)})

p <- forest_plot(model_list = univ_models, model_names = c("T_Inflamed", paste0("Stabl_top",topn_range), paste0("T_Inflamed_Stabl_top",topn_range)),
                 xlim = c(0, 2.5), ticks_at = c(0.5, 1, 1.5, 2, 2.5))
                   
p_sc <- get_scale(plot = p, width_wanted = 10, height_wanted = 8, unit = "in")
ggsave(filename = "Figure/SFigure6B.pdf", plot = p, width = 10, height = 8, scale = p_sc)

# p <- forest_model(model_list = univ_models,  merge_models =T)
# ggsave("Figure/SFigure6B_old.pdf", p, width=8, height=8)

In [33]:
# Compute combined models for the T cell inflamed GEP + immune-altered signatures

topn_range <- c(10,15,20,25,30)

lrt_comp <- data.frame(matrix(0, nrow = length(topn_range), ncol=2))
rownames(lrt_comp) <- paste0("Stabl_top",topn_range)
colnames(lrt_comp) <- c("T_Inflamed","Immune_altered")


pname <- "SU2C"


tested_studies <- c("NSCLC_Advanced_GSE135222_Kim2020","NSCLC_Metastatic_GSE136961_Hwang2020",
                    "STAD_Metastatic_PRJEB25780_Kim2018","Melanoma_Metastatic_PRJEB23709_Gide2019","Melanoma_Advanced_GSE91061_Riaz2017",
                    "Melanoma_Metastatic_GSE78220_Hugo2016")

study[[pname]]$stabl$val <- list()

for(pname2 in tested_studies){
    
    study[[pname]]$stabl$val[[pname2]] <- list()

    topn_uni <- data.frame(matrix(NA, ncol=4, nrow=length(topn_range)))
    rownames(topn_uni) <- paste0("Stabl_top",topn_range)
    colnames(topn_uni) <- c("beta","p","AIC","AUC")
    topn_uni$beta <- 0
    topn_uni$p <- 1
    
    topn_val <- topn_uni

    topn_pair <- data.frame(matrix(NA, ncol=6, nrow=length(topn_range)))
    rownames(topn_pair) <- paste0("Stabl_top",topn_range)
    colnames(topn_pair) <- c("b1","b2","p1","p2","AIC","AUC")
    topn_pair$b1 <- 0
    topn_pair$b2 <- 0
    topn_pair$p1 <- 1
    topn_pair$p2 <- 1
    
    topn_lrt <- lrt_comp

    for(topn in topn_range){
        study[[pname2]]$pheno[[paste0("Stabl_top",topn)]] <- 0
        valid_genes <- intersect(study[[pname]]$stabl[[paste0("Stabl_top",topn)]], rownames(study[[pname2]]$exp))
        if(length(valid_genes) == 0){
            next
        }
        
        study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]] <- list()
        study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]]$genes <- valid_genes
        
        temp_pheno <- data.frame(
            Response_Bin = study[[pname2]]$pheno$Response_Bin,
            T_Inflamed = study[[pname2]]$pheno$T_Inflamed,
            Test = 0
        )
        
        if(length(valid_genes) == 1){
            temp_pheno$Test <- as.numeric(scale(as.numeric(study[[pname2]]$exp[valid_genes,])))
        } else {
            temp_pheno$Test <- rowMeans(scale(t(study[[pname2]]$exp[valid_genes,])))
        }
        study[[pname2]]$pheno[[paste0("Stabl_top",topn)]] <- temp_pheno$Test
        
        lr.t <- glm(as.formula(paste0("Response_Bin ~ T_Inflamed")), family = "binomial", data = temp_pheno)
        
        # combine T_Inflamed and immune-altered signatures using weights learned from SU2C
        temp_pheno$CombinedVal <- temp_pheno$T_Inflamed*study[[pname]]$stabl$topn_pair[paste0("Stabl_top",topn), "b1"] +
            temp_pheno$Test*study[[pname]]$stabl$topn_pair[paste0("Stabl_top",topn), "b2"]
        
        # predict response using immune-altered signature alone
        lr.uni <- glm(as.formula(paste0("Response_Bin ~ Test")), family = "binomial", data = temp_pheno)
        test <- summary(lr.uni)
        if("Test" %in% rownames(test$coefficients)){
            test_ind <- which(paste0("Test") == rownames(test$coefficients))
            topn_uni[paste0("Stabl_top",topn), "beta"] <- test$coefficients[test_ind,1]
            topn_uni[paste0("Stabl_top",topn), "p"] <- test$coefficients[test_ind,4]
        }
        topn_uni[paste0("Stabl_top",topn), "AIC"] <- test$aic
        topn_uni[paste0("Stabl_top",topn), "AUC"] <- auc(temp_pheno$Response_Bin, -temp_pheno$Test, direction="<")
        study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]]$uni <- test
        
        # predict response using T_Inflamed+immune-altered signature combined by SU2C weights
        test <- summary(glm(as.formula(paste0("Response_Bin ~ CombinedVal")), family = "binomial", data = temp_pheno))
        if("CombinedVal" %in% rownames(test$coefficients)){
            test_ind <- which(paste0("CombinedVal") == rownames(test$coefficients))
            topn_val[paste0("Stabl_top",topn), "beta"] <- test$coefficients[test_ind,1]
            topn_val[paste0("Stabl_top",topn), "p"] <- test$coefficients[test_ind,4]
        }
        topn_val[paste0("Stabl_top",topn), "AIC"] <- test$aic
        topn_val[paste0("Stabl_top",topn), "AUC"] <- auc(temp_pheno$Response_Bin, temp_pheno$CombinedVal, direction="<")
        study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]]$val <- test

        # predict response using T_Inflamed+immune-altered signature
        lr.comb <- glm(as.formula("Response_Bin ~ T_Inflamed + Test"), family = "binomial", data = temp_pheno)
        test <- summary(lr.comb)
        if("T_Inflamed" %in% rownames(test$coefficients)){
            test_ind <- which("T_Inflamed" == rownames(test$coefficients))
            topn_pair[paste0("Stabl_top",topn), "b1"] <- test$coefficients[test_ind,1]
            topn_pair[paste0("Stabl_top",topn), "p1"] <- test$coefficients[test_ind,4]
        }
        if("Test" %in% rownames(test$coefficients)){
            test_ind <- which("Test" == rownames(test$coefficients))
            topn_pair[paste0("Stabl_top",topn), "b2"] <- test$coefficients[test_ind,1]
            topn_pair[paste0("Stabl_top",topn), "p2"] <- test$coefficients[test_ind,4]
        }
        topn_pair[paste0("Stabl_top",topn), "AIC"] <- test$aic
        temp_pheno$Combined <- temp_pheno$T_Inflamed*topn_pair[paste0("Stabl_top",topn), "b1"] +
            temp_pheno$Test*topn_pair[paste0("Stabl_top",topn), "b2"]

        topn_pair[paste0("Stabl_top",topn), "AUC"] <- auc(temp_pheno$Response_Bin, temp_pheno$Combined, direction="<")
        study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]]$pair <- test
        
        topn_lrt[paste0("Stabl_top",topn), "T_Inflamed"] <- anova(lr.t,lr.comb,test="LRT")[2,"Pr(>Chi)"]
        topn_lrt[paste0("Stabl_top",topn), "Immune_altered"] <- anova(lr.uni,lr.comb,test="LRT")[2,"Pr(>Chi)"]
    }

    study[[pname]]$stabl$val[[pname2]]$topn_uni <- topn_uni
    study[[pname]]$stabl$val[[pname2]]$topn_val <- topn_val
    study[[pname]]$stabl$val[[pname2]]$topn_pair <- topn_pair
    study[[pname]]$stabl$val[[pname2]]$topn_lrt <- topn_lrt
}

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Setting levels: control = 0, case = 1

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Setting levels: control = 0, case = 1

Setting levels: control = 0

In [34]:
# SFigure 7 and 8

pname <- "SU2C"

topn_range <- c(10,15,20,25,30)


# P-value changes

for(pname2 in names(study[[pname]]$stabl$val)){
    name_vec <- unlist(strsplit(pname2,"_"))
    study_name <- paste0(name_vec[1],"_",name_vec[4])
    
    if(study_name == "STAD_Kim2018"){
        study_name <- "GC_Kim2018"
    }
    
    T_inf_label <- paste0("T_Inflamed (n=",length(intersect(rownames(study[[pname2]]$exp), icb_gsdb$T_Inflamed)),")")
    topn_label <- c()
    for(topn in topn_range){
        if(is.null(study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]])){
            topn_label <- c(topn_label, paste0("Top",topn," (n=0)"))
        } else {
            topn_label <- c(topn_label, paste0("Top",topn," (n=",length(study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]]$genes),")"))
        }
    }
    
    dat.plot <- data.frame(matrix(NA, ncol=5, nrow=2*length(topn_range)+1))
    colnames(dat.plot) <- c("pval","beta","Type","Signature","Predictor")
    
    dat.plot[1, "pval"] <- study[[pname2]]$uni_gs["T_Inflamed","p"]
    dat.plot[1, "beta"] <- study[[pname2]]$uni_gs["T_Inflamed","beta"]
    dat.plot[1, "Signature"] <- T_inf_label
    dat.plot[1, "Type"] <- "Signature alone"
    dat.plot[1, "Predictor"] <- "T_Inflamed"
    
    dat.plot[2:(1+length(topn_range)), "pval"] <- study[[pname]]$stabl$val[[pname2]]$topn_uni$p
    dat.plot[2:(1+length(topn_range)), "beta"] <- study[[pname]]$stabl$val[[pname2]]$topn_uni$beta
    dat.plot[2:(1+length(topn_range)), "Signature"] <- topn_label
    dat.plot[2:(1+length(topn_range)), "Type"] <- "Signature alone"
    dat.plot[2:(1+length(topn_range)), "Predictor"] <- paste0("Top",topn_range)
    
    dat.plot[(1+length(topn_range)+1):(1+2*length(topn_range)), "pval"] <- study[[pname]]$stabl$val[[pname2]]$topn_val$p
    dat.plot[(1+length(topn_range)+1):(1+2*length(topn_range)), "beta"] <- study[[pname]]$stabl$val[[pname2]]$topn_val$beta
    dat.plot[(1+length(topn_range)+1):(1+2*length(topn_range)), "Signature"] <- topn_label
    dat.plot[(1+length(topn_range)+1):(1+2*length(topn_range)), "Type"] <- "Weighted difference"
    dat.plot[(1+length(topn_range)+1):(1+2*length(topn_range)), "Predictor"] <- paste0("T_Inf+Top",topn_range,"\n(weighted_diff)")
    
    dat.plot$Predictor <- factor(dat.plot$Predictor, levels=rev(c("T_Inflamed",paste0("Top",topn_range),paste0("T_Inf+Top",topn_range,"\n(weighted_diff)"))))
    dat.plot$SignedLog10P <- -log10(dat.plot$pval)*sign(dat.plot$beta)
    p <- ggplot(data = dat.plot, aes(x=Predictor, y=SignedLog10P, fill=Signature)) + geom_bar(stat="identity") +
        geom_hline(yintercept = c(-log10(0.05), 0, log10(0.05)), linetype=c("dashed","solid","dashed"), color=c("blue","darkgrey","blue")) +
        scale_fill_manual(values = c("black", RColorBrewer::brewer.pal(n=5,"Set2"))) + scale_color_manual(values = c()) +
        labs(title=study_name) + theme_bw() + coord_flip()
    ggsave(paste0("Figure/SFigure7_8_",pname2,".pdf"), p, width=6, height=4)
}

In [35]:
# SFigure 10 and 12

pname <- "SU2C"

topn_range <- c(10,15,20,25,30)

# P-value changes

for(pname2 in names(study[[pname]]$stabl$val)){
    name_vec <- unlist(strsplit(pname2,"_"))
    study_name <- paste0(name_vec[1],"_",name_vec[4])
    
    if(study_name == "STAD_Kim2018"){
        study_name <- "GC_Kim2018"
    }
    
    T_inf_label <- paste0("T_Inflamed (n=",length(intersect(rownames(study[[pname2]]$exp), icb_gsdb$T_Inflamed)),")")
    topn_label <- c()
    for(topn in topn_range){
        if(is.null(study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]])){
            topn_label <- c(topn_label, paste0("Top",topn," (n=0)"))
        } else {
            topn_label <- c(topn_label, paste0("Top",topn," (n=",length(study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]]$genes),")"))
        }
    }
    
    dat.plot <- data.frame(matrix(NA, ncol=6, nrow=4*length(topn_range)))
    colnames(dat.plot) <- c("pval","beta","Model","Group","Signature","Type")
    
    dat.plot[1:length(topn_range), "pval"] <- study[[pname2]]$uni_gs["T_Inflamed","p"]
    dat.plot[1:length(topn_range), "beta"] <- study[[pname2]]$uni_gs["T_Inflamed","beta"]
    dat.plot[1:length(topn_range), "Model"] <- "Univariate"
    dat.plot[1:length(topn_range), "Group"] <- paste0("T_Inflamed:","Top",topn_range)
    dat.plot[1:length(topn_range), "Signature"] <- T_inf_label
    dat.plot[1:length(topn_range), "Type"] <- topn_label
    
    dat.plot[(1+length(topn_range)):(2*length(topn_range)), "pval"] <- study[[pname]]$stabl$val[[pname2]]$topn_uni$p
    dat.plot[(1+length(topn_range)):(2*length(topn_range)), "beta"] <- study[[pname]]$stabl$val[[pname2]]$topn_uni$beta
    dat.plot[(1+length(topn_range)):(2*length(topn_range)), "Model"] <- "Univariate"
    dat.plot[(1+length(topn_range)):(2*length(topn_range)), "Group"] <- paste0("Top",topn_range,":","Top",topn_range)
    dat.plot[(1+length(topn_range)):(2*length(topn_range)), "Signature"] <- "Immune_altered"
    dat.plot[(1+length(topn_range)):(2*length(topn_range)), "Type"] <- topn_label
    
    dat.plot[(2*length(topn_range)+1):(length(topn_range)*3), "pval"] <- study[[pname]]$stabl$val[[pname2]]$topn_pair$p1
    dat.plot[(2*length(topn_range)+1):(length(topn_range)*3), "beta"] <- study[[pname]]$stabl$val[[pname2]]$topn_pair$b1
    dat.plot[(2*length(topn_range)+1):(length(topn_range)*3), "Model"] <- "Combined"
    dat.plot[(2*length(topn_range)+1):(length(topn_range)*3), "Group"] <- paste0("T_Inflamed:","Top",topn_range)
    dat.plot[(2*length(topn_range)+1):(length(topn_range)*3), "Signature"] <- T_inf_label
    dat.plot[(2*length(topn_range)+1):(length(topn_range)*3), "Type"] <- topn_label
    
    dat.plot[(3*length(topn_range)+1):(length(topn_range)*4), "pval"] <- study[[pname]]$stabl$val[[pname2]]$topn_pair$p2
    dat.plot[(3*length(topn_range)+1):(length(topn_range)*4), "beta"] <- study[[pname]]$stabl$val[[pname2]]$topn_pair$b2
    dat.plot[(3*length(topn_range)+1):(length(topn_range)*4), "Model"] <- "Combined"
    dat.plot[(3*length(topn_range)+1):(length(topn_range)*4), "Group"] <- paste0("Top",topn_range,":","Top",topn_range)
    dat.plot[(3*length(topn_range)+1):(length(topn_range)*4), "Signature"] <- "Immune_altered"
    dat.plot[(3*length(topn_range)+1):(length(topn_range)*4), "Type"] <- topn_label
    
    dat.plot$Model <- factor(dat.plot$Model, levels=c("Univariate","Combined"))
    dat.plot$SignedLog10P <- -log10(dat.plot$pval)*sign(dat.plot$beta)
    p <- ggplot(data = dat.plot, aes(x=Model, y=SignedLog10P, group=Group)) + geom_line(aes(color=Type)) +
        geom_point(aes(shape=Signature), size=2) + labs(title=study_name) +
        geom_hline(yintercept = c(-log10(0.05), 0, log10(0.05)), linetype=c("dashed","solid","dashed"), color=c("blue","darkgrey","blue")) +
        theme_bw() + scale_color_manual(values = c(RColorBrewer::brewer.pal(n=5,"Set2")))
    ggsave(paste0("Figure/SFigure10_12_",pname2,".pdf"), p, width=6, height=4)
}

In [36]:
# Figure 6 and SFigure 9

pname <- "SU2C"

topn_range <- c(10,15,20,25,30)

for(pname2 in names(study[[pname]]$stabl$val)){
    name_vec <- unlist(strsplit(pname2,"_"))
    study_name <- paste0(name_vec[1],"_",name_vec[4])
    
    if(study_name == "STAD_Kim2018"){
        study_name <- "GC_Kim2018"
    }
    
    T_inf_label <- paste0("T_Inflamed (n=",length(intersect(rownames(study[[pname2]]$exp), icb_gsdb$T_Inflamed)),")")
    topn_label <- c()
    for(topn in topn_range){
        if(is.null(study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]])){
            topn_label <- c(topn_label, paste0("Top",topn," (n=0)"))
        } else {
            topn_label <- c(topn_label, paste0("Top",topn," (n=",length(study[[pname]]$stabl$val[[pname2]][[paste0("Stabl_top",topn)]]$genes),")"))
        }
    }
    
    dat.plot <- data.frame(
        Model = c(rep("Univariate", length(topn_range)+1), rep("Combined\n(Trained_Weight)", length(topn_range)), rep("Univariate\n(SU2C_Weight)", length(topn_range))),
        AIC = c(study[[pname2]]$uni_gs["T_Inflamed","AIC"], study[[pname]]$stabl$val[[pname2]]$topn_uni$AIC, study[[pname]]$stabl$val[[pname2]]$topn_pair$AIC, study[[pname]]$stabl$val[[pname2]]$topn_val$AIC),
        AUC = c(study[[pname2]]$uni_gs["T_Inflamed","AUC"], study[[pname]]$stabl$val[[pname2]]$topn_uni$AUC, study[[pname]]$stabl$val[[pname2]]$topn_pair$AUC, study[[pname]]$stabl$val[[pname2]]$topn_val$AUC),
        TopGene = c(T_inf_label, topn_label, topn_label, topn_label)
    )
    dat.plot$Model <- factor(dat.plot$Model, levels = c("Univariate","Univariate\n(SU2C_Weight)","Combined\n(Trained_Weight)"))

    p <- ggplot(data = dat.plot, aes(x=Model, y=AIC, group=TopGene, color=TopGene)) + geom_point() + geom_line() + theme_bw() +
        labs(title=study_name) + scale_color_manual(values = c("black", RColorBrewer::brewer.pal(n=5,"Set2"))) +
        geom_hline(yintercept = study[[pname2]]$uni_gs["T_Inflamed","AIC"], linetype="dashed", color="black")
    ggsave(paste0("Figure/Figure6ABC_SFigure9ABC_",pname2,".pdf"), p, width=6, height=3)

    p <- ggplot(data = dat.plot, aes(x=Model, y=AUC, group=TopGene, color=TopGene)) + geom_point() + geom_line() + labs(title=study_name) +
        geom_hline(yintercept = sort(study[[pname2]]$rand_model$auc_randg)[95], linetype="dashed", color="darkgrey") + theme_bw() +
        scale_color_manual(values = c("black", RColorBrewer::brewer.pal(n=5,"Set2")))
    ggsave(paste0("Figure/Figure6DEF_SFigure9DEF_",pname2,".pdf"), p, width=6, height=3)
}

Warning message:
“Removed 3 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 3 rows containing missing values (`geom_line()`).”
Warning message:
“Removed 3 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 3 rows containing missing values (`geom_line()`).”


In [37]:
# SFigure 11 and 13

pname <- "SU2C"

topn_range <- c(10,15,20,25,30)

for(pname2 in names(study[[pname]]$stabl$val)){
    name_vec <- unlist(strsplit(pname2,"_"))
    study_name <- paste0(name_vec[1],"_",name_vec[4])
    
    if(study_name == "STAD_Kim2018"){
        study_name <- "GC_Kim2018"
    }

    dat.plot <- study[[pname]]$stabl$val[[pname2]]$topn_lrt
    
    if(any(dat.plot == 0)){
        dat.plot[dat.plot == 0] <- NA
    }
    dat.plot <- -log10(dat.plot)
    dat.plot$Model <- paste0("Response ~ T_Inflamed + ", paste0("Stabl_top",topn_range))


    p1 <- ggplot(data = dat.plot, aes(x = Model, y = T_Inflamed)) + geom_bar(stat = "identity") + geom_hline(yintercept = -log10(0.05), color = "blue", linetype="dashed") +
        labs(title = "Response ~ T_Inflamed", y = "-Log10P (LRT test)") + coord_flip() + theme_bw()
    p2 <- ggplot(data = dat.plot, aes(x = Model, y = Immune_altered)) + geom_bar(stat = "identity") + geom_hline(yintercept = -log10(0.05), color = "blue", linetype="dashed") +
        labs(title = "Response ~ Immune-altered signature", y = "-Log10P (LRT test)") + coord_flip() + theme_bw() + theme(axis.text.y = element_blank(), axis.title.y = element_blank())
    ggsave(paste0("Figure/SFigure11_13_",study_name,".pdf"), p1+p2, width = 10, height = 3)
}

Warning message:
“Removed 1 rows containing missing values (`position_stack()`).”
Warning message:
“Removed 1 rows containing missing values (`position_stack()`).”


In [34]:
# Supplementary tables for model details

topn_range <- c(10,15,20,25,30)

out_cols <- c("sid","Response_Bin","Harmonized_Confirmed_BOR_3_Cat","Harmonized_OS_Months", "Harmonized_OS_Event","Harmonized_PFS_Months","Harmonized_PFS_Event","M_cluster",
              "Sex","Smoking_Status","Histology","PDL1_TPS","Age_at_Diagnosis","Line_of_Therapy",rownames(study$SU2C$uni_gs),paste0("Stabl_top",topn_range))

temp <- study$SU2C$pheno[, out_cols]
write.csv(temp, "Table/su2c_pheno_all_signature.csv", row.names=F)

temp <- study$SU2C$pair_neg$stat[,c(1:6,9:10)]
write.csv(temp, "Table/su2c_GEP_myeloid.csv", row.names=F)
temp <- rbind(study$SU2C$uni_gs["T_Inflamed",],study$SU2C$uni_gs[study$SU2C$pair_neg$gs,])
write.csv(temp, "Table/su2c_myeloid_uni.csv", row.names=T)

temp <- cbind(data.frame(v1="T_Inflamed",v2=c(rownames(study$SU2C$stabl$topn_pair))),study$SU2C$stabl$topn_pair)
write.csv(temp, "Table/su2c_GEP_stabl.csv", row.names=F)
temp <- rbind(study$SU2C$uni_gs["T_Inflamed",], study$SU2C$stabl$topn_uni)
write.csv(temp, "Table/su2c_stabl_uni.csv", row.names=T)

pname <- "NSCLC_Advanced_GSE135222_Kim2020"
temp <- cbind(data.frame(v1="T_Inflamed",v2=c(rownames(study$SU2C$stabl$val[[pname]]$topn_pair))),study$SU2C$stabl$val[[pname]]$topn_pair)
write.csv(temp, "Table/Kim2020_GEP_stabl.csv", row.names=F)
temp <- rbind(study[[pname]]$uni_gs["T_Inflamed",],study$SU2C$stabl$val[[pname]]$topn_uni)
write.csv(temp, "Table/Kim2020_uni.csv", row.names=T)
temp <- study$SU2C$stabl$val[[pname]]$topn_val
write.csv(temp, "Table/Kim2020_uni_su2c.csv", row.names=T)

pname <- "NSCLC_Metastatic_GSE136961_Hwang2020"
temp <- cbind(data.frame(v1="T_Inflamed",v2=c(rownames(study$SU2C$stabl$val[[pname]]$topn_pair))),study$SU2C$stabl$val[[pname]]$topn_pair)
write.csv(temp, "Table/Hwang2020_GEP_stabl.csv", row.names=F)
temp <- rbind(study[[pname]]$uni_gs["T_Inflamed",],study$SU2C$stabl$val[[pname]]$topn_uni)
write.csv(temp, "Table/Hwang2020_uni.csv", row.names=T)
temp <- study$SU2C$stabl$val[[pname]]$topn_val
write.csv(temp, "Table/Hwang2020_uni_su2c.csv", row.names=T)

pname <- "STAD_Metastatic_PRJEB25780_Kim2018"
temp <- cbind(data.frame(v1="T_Inflamed",v2=c(rownames(study$SU2C$stabl$val[[pname]]$topn_pair))),study$SU2C$stabl$val[[pname]]$topn_pair)
write.csv(temp, "Table/Kim2018_GEP_stabl.csv", row.names=F)
temp <- rbind(study[[pname]]$uni_gs["T_Inflamed",],study$SU2C$stabl$val[[pname]]$topn_uni)
write.csv(temp, "Table/Kim2018_uni.csv", row.names=T)
temp <- study$SU2C$stabl$val[[pname]]$topn_val
write.csv(temp, "Table/Kim2018_uni_su2c.csv", row.names=T)

pname <- "Melanoma_Metastatic_PRJEB23709_Gide2019"
temp <- cbind(data.frame(v1="T_Inflamed",v2=c(rownames(study$SU2C$stabl$val[[pname]]$topn_pair))),study$SU2C$stabl$val[[pname]]$topn_pair)
write.csv(temp, "Table/Gide2019_GEP_stabl.csv", row.names=F)
temp <- rbind(study[[pname]]$uni_gs["T_Inflamed",],study$SU2C$stabl$val[[pname]]$topn_uni)
write.csv(temp, "Table/Gide2019_uni.csv", row.names=T)
temp <- study$SU2C$stabl$val[[pname]]$topn_val
write.csv(temp, "Table/Gide2019_uni_su2c.csv", row.names=T)

pname <- "Melanoma_Advanced_GSE91061_Riaz2017"
temp <- cbind(data.frame(v1="T_Inflamed",v2=c(rownames(study$SU2C$stabl$val[[pname]]$topn_pair))),study$SU2C$stabl$val[[pname]]$topn_pair)
write.csv(temp, "Table/Riaz2017_GEP_stabl.csv", row.names=F)
temp <- rbind(study[[pname]]$uni_gs["T_Inflamed",],study$SU2C$stabl$val[[pname]]$topn_uni)
write.csv(temp, "Table/Riaz2017_uni.csv", row.names=T)
temp <- study$SU2C$stabl$val[[pname]]$topn_val
write.csv(temp, "Table/Riaz2017_uni_su2c.csv", row.names=T)

pname <- "Melanoma_Metastatic_GSE78220_Hugo2016"
temp <- cbind(data.frame(v1="T_Inflamed",v2=c(rownames(study$SU2C$stabl$val[[pname]]$topn_pair))),study$SU2C$stabl$val[[pname]]$topn_pair)
write.csv(temp, "Table/Hugo2016_GEP_stabl.csv", row.names=F)
temp <- rbind(study[[pname]]$uni_gs["T_Inflamed",],study$SU2C$stabl$val[[pname]]$topn_uni)
write.csv(temp, "Table/Hugo2016_uni.csv", row.names=T)
temp <- study$SU2C$stabl$val[[pname]]$topn_val
write.csv(temp, "Table/Hugo2016_uni_su2c.csv", row.names=T)


In [24]:
# Supplementary table to demonstrate improvements of combined models

n_r <- sum(study$SU2C$pheno$Response_Bin == 1)
n_nr <- sum(study$SU2C$pheno$Response_Bin == 0)
med <- as.integer((nrow(study$SU2C$pheno)+1)/2)

temp <- data.frame(matrix(NA, ncol=7, nrow=1+nrow(study$SU2C$pair_neg$stat)))
colnames(temp) <- c("Signature","TP","TPR","TN","TNR","ACC","AUC")

temp[1, "Signature"] <- "T_Inflamed"
temp[1, "TP"] <- sum(study$SU2C$pheno$Response_Bin[order(study$SU2C$pheno$T_Inflamed, decreasing = T)[1:med]] == 1)
temp[1, "TN"] <- sum(study$SU2C$pheno$Response_Bin[order(study$SU2C$pheno$T_Inflamed, decreasing = T)[(med+1):(n_r+n_nr)]] == 0)
temp[1, "TPR"] <- temp[1, "TP"]/med
temp[1, "TNR"] <- temp[1, "TN"]/med
temp[1, "ACC"] <- (temp[1, "TP"]+temp[1, "TN"])/(n_r+n_nr)
temp[1, "AUC"] <- study$SU2C$uni_gs["T_Inflamed", "AUC"]

for(i in 1:nrow(study$SU2C$pair_neg$stat)){
    temp_pred <- study$SU2C$pair_neg$stat[i, "b1"]*study$SU2C$pheno[[study$SU2C$pair_neg$stat[i, "v1"]]]+
        study$SU2C$pair_neg$stat[i, "b2"]*study$SU2C$pheno[[study$SU2C$pair_neg$stat[i, "v2"]]]
    
    temp[1+i, "Signature"] <- paste0(study$SU2C$pair_neg$stat[i, "v2"],"+",study$SU2C$pair_neg$stat[i, "v1"])
    temp[1+i, "TP"] <- sum(study$SU2C$pheno$Response_Bin[order(temp_pred, decreasing = T)[1:med]] == 1)
    temp[1+i, "TN"] <- sum(study$SU2C$pheno$Response_Bin[order(temp_pred, decreasing = T)[(med+1):(n_r+n_nr)]] == 0)
    temp[1+i, "TPR"] <- temp[1+i, "TP"]/med
    temp[1+i, "TNR"] <- temp[1+i, "TN"]/med
    temp[1+i, "ACC"] <- (temp[1+i, "TP"]+temp[1+i, "TN"])/(n_r+n_nr)
    temp[1+i, "AUC"] <- study$SU2C$pair_neg$stat[i, "AUC"]
}

temp$TPR_inc <- temp$TPR - temp[1,"TPR"]
temp$TNR_inc <- temp$TNR - temp[1,"TNR"]
temp$ACC_inc <- temp$ACC - temp[1,"ACC"]
temp$predicted <- med
temp$total <- n_r+n_nr

temp <- temp[,c("Signature","TP","predicted","total","TPR","TPR_inc")]
colnames(temp) <- c("Model","nTP","nPos","nTotal","TPR (Response Rate)","TPR Increase (vs. T_Inflamed)")

write.csv(temp, "Table/improvement_myeloid_SU2C.csv", row.names=F)


topn_range <- c(10,15,20,25,30)
for(pname2 in names(study$SU2C$stabl$val)){
    n_r <- sum(study[[pname2]]$pheno$Response_Bin == 1)
    n_nr <- sum(study[[pname2]]$pheno$Response_Bin == 0)
    med <- as.integer((nrow(study[[pname2]]$pheno)+1)/2)

    temp <- data.frame(matrix(NA, ncol=7, nrow=1+length(topn_range)))
    colnames(temp) <- c("Signature","TP","TPR","TN","TNR","ACC","AUC")
    rownames(temp) <- c("T_Inflamed", paste0("T_Stabl_top",topn_range))

    temp["T_Inflamed", "Signature"] <- "T_Inflamed"
    temp["T_Inflamed", "TP"] <- sum(study[[pname2]]$pheno$Response_Bin[order(study[[pname2]]$pheno$T_Inflamed, decreasing = T)[1:med]] == 1)
    temp["T_Inflamed", "TN"] <- sum(study[[pname2]]$pheno$Response_Bin[order(study[[pname2]]$pheno$T_Inflamed, decreasing = T)[(med+1):(n_r+n_nr)]] == 0)
    temp["T_Inflamed", "TPR"] <- temp[1, "TP"]/med
    temp["T_Inflamed", "TNR"] <- temp[1, "TN"]/med
    temp["T_Inflamed", "ACC"] <- (temp[1, "TP"]+temp[1, "TN"])/(n_r+n_nr)
    temp["T_Inflamed", "AUC"] <- study[[pname2]]$uni_gs["T_Inflamed", "AUC"]

    for(topn in topn_range){
        temp_pred <- study$SU2C$stabl$val[[pname2]]$topn_pair[paste0("Stabl_top",topn), "b1"]*study[[pname2]]$pheno[["T_Inflamed"]]+
            study$SU2C$stabl$val[[pname2]]$topn_pair[paste0("Stabl_top",topn), "b2"]*study[[pname2]]$pheno[[paste0("Stabl_top",topn)]]
        
        if(all(temp_pred == 0)) temp_pred <- study[[pname2]]$pheno[["T_Inflamed"]]

        temp[paste0("T_Stabl_top",topn), "Signature"] <- paste0("T_Inflamed + Stabl_top",topn)
        temp[paste0("T_Stabl_top",topn), "TP"] <- sum(study[[pname2]]$pheno$Response_Bin[order(temp_pred, decreasing = T)[1:med]] == 1)
        temp[paste0("T_Stabl_top",topn), "TN"] <- sum(study[[pname2]]$pheno$Response_Bin[order(temp_pred, decreasing = T)[(med+1):(n_r+n_nr)]] == 0)
        temp[paste0("T_Stabl_top",topn), "TPR"] <- temp[paste0("T_Stabl_top",topn), "TP"]/med
        temp[paste0("T_Stabl_top",topn), "TNR"] <- temp[paste0("T_Stabl_top",topn), "TN"]/med
        temp[paste0("T_Stabl_top",topn), "ACC"] <- (temp[paste0("T_Stabl_top",topn), "TP"]+temp[paste0("T_Stabl_top",topn), "TN"])/(n_r+n_nr)
        temp[paste0("T_Stabl_top",topn), "AUC"] <- study$SU2C$stabl$val[[pname2]]$topn_uni[paste0("T_Stabl_top",topn), "AUC"]
    }

    temp$TPR_inc <- temp$TPR - temp["T_Inflamed","TPR"]
    temp$TNR_inc <- temp$TNR - temp["T_Inflamed","TNR"]
    temp$ACC_inc <- temp$ACC - temp["T_Inflamed","ACC"]
    temp$predicted <- med
    temp$total <- n_r+n_nr

    temp <- temp[,c("Signature","TP","predicted","total","TPR","TPR_inc")]
    colnames(temp) <- c("Model","nTP","nPos","nTotal","TPR (Response Rate)","TPR Increase (vs. T_Inflamed)")

    write.csv(temp, paste0("Table/improvement_Stabl_",pname2,".csv"), row.names=F)
}

In [40]:
# supplementary table for 170 signatures

temp <- data.frame(Name = names(icb_gsdb), Category = stringr::str_split_fixed(names(icb_gsdb),"_",2)[,1], PMID = "", Reference = "")
write.csv(temp, "Table/signature_info.csv", row.names = F)

In [1]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.7.1 
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.7.1

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] digest_0.6.34   IRdisplay_1.1   utf8_1.2.4      base64enc_0.1-3
 [5] fastmap_1.1.1   glue_1.7.0      htmltools_0.5.7 repr_1.1.6     
 [9] lifecycle_1.0.4 cli_3.6.2       fansi_1.0.6     vctrs_0.